# Graph Convolutional Networks

* graph G = (E, V)
* inputs
    * **X**: input feature matrix  of shape (N x F) or (Number of Nodes x number of input features)
    * **A**: adjacency matrix of size (N x N)
* **Hidden layer**: Hⁱ = f(Hⁱ = f(Hⁱ⁻¹, A)) 
    * H⁰ = X
    * Hⁱ is a N × Fⁱ feature matrix where each row is a feature representation of a node
    * feature become increasingly more abstract after the features are aggregated at each layer 
    * e.g. f(Hⁱ, A) = σ(AHⁱWⁱ) where sigma is a RELU function and Wⁱ is the weight matrix for layer i (dim  Fⁱ × Fⁱ⁺¹)
    


In [ ]:
import tensorflow as tf
import networkx
import numpy as np

In [44]:
# Adjacency Matrix A
A = np.matrix([
        [0, 1, 0, 0], 
        [0, 0, 1, 1], 
        [0, 1, 0, 0], 
        [1, 0, 1, 0]
    ], dtype = float)
print (A)
print ("shape of A: [%d, %d]" % (A.shape))

[[ 0.  1.  0.  0.]
 [ 0.  0.  1.  1.]
 [ 0.  1.  0.  0.]
 [ 1.  0.  1.  0.]]
shape of A: [4, 4]


In [17]:
# Feature Matrix X
X = np.matrix([[i, -i] for i in range(A.shape[0])], dtype = float)
print (X)
print ("shape of X: [%d, %d]" % (X.shape))

[[ 0.  0.]
 [ 1. -1.]
 [ 2. -2.]
 [ 3. -3.]]
shape of X: [4, 2]


In [28]:
#  f(X, A) = AX where sigma = identify function (multiply by 1)
print (A * X) # also can be written as np.dot(A, X)

# The representation of each node (each row) is 
#     now a sum of its neighbors features

[[ 1. -1.]
 [ 5. -5.]
 [ 1. -1.]
 [ 2. -2.]]


In [23]:
# Problem 1: the aggregated representation AX doesn't include own features
I = np.eye(A.shape[0])
A_hat = A + I
print (A_hat)

[[ 1.  1.  0.  0.]
 [ 0.  1.  1.  1.]
 [ 0.  1.  1.  0.]
 [ 1.  0.  1.  1.]]


In [25]:
# Now own features are also added because the nodes are neighbors of itself
print(A_hat * X)

[[ 1. -1.]
 [ 6. -6.]
 [ 3. -3.]
 [ 5. -5.]]


In [39]:
# Problem 2: nodes with large degrees will have large values, 
# and nodes with small degrees will have small values
# ==> need to normalize values of A by multiply A with inverse degree matrix D

D = np.array(np.sum(A, axis = 0))[0] # degrees of each node
print (D)
D = np.matrix(np.diag(D))
print ('=================================')
print (D)

[ 1.  2.  2.  1.]
[[ 1.  0.  0.  0.]
 [ 0.  2.  0.  0.]
 [ 0.  0.  2.  0.]
 [ 0.  0.  0.  1.]]


In [38]:
# Divide the values in each row of the adjacency matrix by the degree of the node
print (D**-1)
print ('=================================')
print(D**-1 * A)

[[ 1.   0.   0.   0. ]
 [ 0.   0.5  0.   0. ]
 [ 0.   0.   0.5  0. ]
 [ 0.   0.   0.   1. ]]
[[ 0.   1.   0.   0. ]
 [ 0.   0.   0.5  0.5]
 [ 0.   0.5  0.   0. ]
 [ 1.   0.   1.   0. ]]


In [47]:
# apply propogation rule with the transformed adjacency matrix
# get node representations corresponding to the mean of the features of neighboring nodes
print ("Before: \n", A * X)
print ("=================================")
print("After: \n", D**-1 * A * X)


Before: 
 [[ 1. -1.]
 [ 5. -5.]
 [ 1. -1.]
 [ 2. -2.]]
After: 
 [[ 1.  -1. ]
 [ 2.5 -2.5]
 [ 0.5 -0.5]
 [ 2.  -2. ]]


## Including self hops:

In [42]:
W = np.matrix([
    [1, -1], [-1, 1]
])
D_hat = np.array(np.sum(A_hat, axis = 0))[0]
D_hat = np.matrix(np.diag(D_hat))
print(D_hat**-1 * A_hat * X * W)

[[ 1. -1.]
 [ 4. -4.]
 [ 2. -2.]
 [ 5. -5.]]


In [48]:
# reduce dimensionality of the output feature:

W = np.matrix([
    [1], [-1]
])
D_hat = np.array(np.sum(A_hat, axis = 0))[0]
D_hat = np.matrix(np.diag(D_hat))
print(D_hat**-1 * A_hat * X * W)

[[ 1.]
 [ 4.]
 [ 2.]
 [ 5.]]


In [50]:
def relu (z):
    a = np.maximum(0, z)
    return a

In [51]:
# RELU: a = max(0, z) 
W = np.matrix([
    [1, -1], [-1, 1]
])
print(relu(D_hat**-1 * A_hat * X * W))

[[ 1.  0.]
 [ 4.  0.]
 [ 2.  0.]
 [ 5.  0.]]
